In [1]:
def load_data_set():
    # 달마시안 애호가 전자 게시판에서 가져온 문서 집합을 토큰화 한 것
    posting_list = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]

    # 1: 폭력적임, 0: 폭력적이지 않음
    # 텍스트에 사람이 직접 분류 항목 표시를 붙이고, 폭력적인 게시물을 자동적으로 감지하는 프로그램을 훈련하는데 사용한다.
    class_vec = [0,1,0,1,0,1] 
    return posting_list, class_vec

# 모든 문서에 있는 모든 유일한 단어 목록을 생성한다.
def create_vocab_list(data_set):
    # 비어있는 집합 생성
    vocab_set = set([]) 
    for document in data_set:
        # 연산자 | 는 두 개의 집합 유형의 변수를 합치는 데 사용하는 연산자.
        # 두 개의 집합 통합 생성
        # ex) {'a', 'b', 'c'} | {'c', 'd', 'e'} 
        #     -> {'a', 'b', 'c', 'd', 'e'} 
        vocab_set = vocab_set | set(document)
    return list(vocab_set)

# 주어진 문서 내에 어휘 목록에 있는 단어가 존재하는지 아닌지를 표현하기 위해 어휘 목록, 문서, 
# 1과 0의 출력 벡터를 사용한다.
def set_of_words_2_vec(vocab_list, input_set):
    # 모두 0인 벡터 생성
    return_vec = [0] * len(vocab_list) 
    for word in input_set:
        if word in vocab_list:
            return_vec[vocab_list.index(word)] = 1
        else:
            print ("the word: %s is not in my Vocablulary!" % word)
    return return_vec

In [2]:
from numpy import *

def train(train_matrix, train_category):
    # 문서의 수
    num_train_docs = len(train_matrix)
    
    # 한문서의 최대 단어수
    num_words = len(train_matrix[0])
    
    # 폭력적인 문서의 확률 계산
    # 1의 값이 폭력적이기 때문에 train_category를 합하면 폭력적인 문서의 수가 나온다.
    # 폭력적인 문서 수 / 전체 문서 수
    # 현재는 분류 항목이 두개 이기때문에 가능 / 분류 항목이 2개 이상이면 이부분을 수정해야 한다.
    p_abusive = sum(train_category) / float(num_train_docs)
    
    # 확률 초기화
    # zerors: num_words(인자값) 수 만큼 0. 배열 만듬
    p_0_num = ones(num_words) 
    p_1_num = ones(num_words)
    p_0_denom = 2.0
    p_1_denom = 2.0
    
    # 벡터 추가 
    for i in range(num_train_docs):
        if train_category[i] == 1:
            p_1_num += train_matrix[i]
            p_1_denom += sum(train_matrix[i])
        else:
            p_0_num += train_matrix[i]
            p_0_denom += sum(train_matrix[i])

    # 원소 나누기
    p_1_vect = log(p_1_num / p_1_denom)
    p_0_vect = log(p_0_num / p_1_denom)
    return p_0_vect, p_1_vect, p_abusive

In [3]:
def classify(vec_2_classify, p_0_vec, p_1_vec, p_class_1):
    p1 = sum(vec_2_classify * p_1_vec) + log(p_class_1)
    p0 = sum(vec_2_classify * p_0_vec) + log(1.0 - p_class_1)

    if p1 > p0:
        return 1
    else:
        return 0

In [4]:
list_o_posts, list_classes = load_data_set()
my_vocab_list = create_vocab_list(list_o_posts)

train_matrix = []

for post_in_doc in list_o_posts:
    train_matrix.append(set_of_words_2_vec(my_vocab_list, post_in_doc))

p_0_v, p_1_v, p_ab = train(array(train_matrix), array(list_classes))

In [5]:
test_entry = ['love', 'my', 'dalmation']
this_doc = array(set_of_words_2_vec(my_vocab_list, test_entry))
print (this_doc, p_0_v, p_1_v, p_ab)
print (test_entry, 'classfied as: ', classify(this_doc, p_0_v, p_1_v, p_ab))

test_entry = ['stupid', 'dog']
this_doc = array(set_of_words_2_vec(my_vocab_list, test_entry))
print (this_doc, p_0_v, p_1_v, p_ab)
print (test_entry, 'classified ad: ', classify(this_doc, p_0_v, p_1_v, p_ab))

[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0] [-3.04452244 -3.04452244 -2.35137526 -2.35137526 -3.04452244 -3.04452244
 -2.35137526 -2.35137526 -2.35137526 -3.04452244 -3.04452244 -2.35137526
 -2.35137526 -2.35137526 -2.35137526 -3.04452244 -2.35137526 -1.94591015
 -2.35137526 -1.65822808 -2.35137526 -2.35137526 -2.35137526 -2.35137526
 -2.35137526 -3.04452244 -3.04452244 -3.04452244 -3.04452244 -2.35137526
 -2.35137526 -2.35137526] [-2.35137526 -2.35137526 -3.04452244 -3.04452244 -1.65822808 -2.35137526
 -3.04452244 -3.04452244 -2.35137526 -2.35137526 -2.35137526 -3.04452244
 -3.04452244 -1.94591015 -3.04452244 -1.94591015 -3.04452244 -2.35137526
 -3.04452244 -3.04452244 -3.04452244 -3.04452244 -2.35137526 -3.04452244
 -3.04452244 -2.35137526 -2.35137526 -2.35137526 -2.35137526 -3.04452244
 -3.04452244 -3.04452244] 0.5
['love', 'my', 'dalmation'] classfied as:  0
[0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [-3.04452244 -3.04452244 -2.35137526 

In [ ]:
list_o_posts, list_classes

In [ ]:
my_vocab_list

In [ ]:
p_0_v